In [4]:
# Import nilearn and numpy
from nilearn import datasets
from nilearn import surface
from nilearn import plotting
from nilearn import image
import numpy as np
import pandas as pd
import glob
from nilearn import signal
from scipy import stats
import nibabel as nib
sk = 3
%matplotlib inline

In [5]:
# List of bold runs
basedir ='/scratch/groups/jyeatman/NSD/ppdata/'
fsdir = '/scratch/groups/jyeatman/NSD/nsddata/freesurfer/'
subs = ['subj01']
runs = ['/func1pt8mm/timeseries/timeseries_session21_run01.nii.gz', '/func1pt8mm/timeseries/timeseries_session21_run14.nii.gz']

In [7]:
for sub in subs:
    fssub = fsdir + sub
    lhsurf = surface.load_surf_mesh(fssub + '/surf/lh.white')
    vwfa = surface.load_surf_data(fssub + '/label/lh.floc-words.mgz')
    curv = surface.load_surf_data(fssub + '/surf/lh.curv')
    vwfa1zmap = np.zeros([lhsurf.coordinates.shape[0], len(runs)])
    vwfa2zmap = np.zeros([lhsurf.coordinates.shape[0], len(runs)])
    r = -1
    for run in runs:
        r = r+1
        boldpath = basedir + sub + run
        bold = image.load_img(boldpath)
        bold = image.smooth_img(bold, sk)
        lhbold = surface.vol_to_surf(img=bold,surf_mesh=lhsurf, radius = 3)
        lhbold = np.transpose(signal.clean(signals = np.transpose(lhbold), filter='cosine',high_pass=0.01, detrend=True, standardize='zscore',t_r=1.333))
        vwfa1seed = np.nanmean(lhbold[vwfa==2],axis=0)
        vwfa2seed = np.nanmean(lhbold[vwfa==3],axis=0)
        vwfa1rval = np.zeros(lhbold.shape[0])
        vwfa2rval = np.zeros(lhbold.shape[0])
        for v in range(lhbold.shape[0]):
            vwfa1rval[v]=np.corrcoef(np.transpose(vwfa1seed), np.transpose(lhbold[v]))[0,1]
            vwfa2rval[v]=np.corrcoef(np.transpose(vwfa2seed), np.transpose(lhbold[v]))[0,1]
        
        vwfa1zmap[:,r] = np.arctanh(vwfa1rval)
        vwfa2zmap[:,r] = np.arctanh(vwfa2rval)

In [ ]:
plotting.plot_surf(lhsurf, surf_map=vwfa, bg_map=curv,threshold = 0.0001,view = 'ventral', cmap = 'jet',colorbar=True)

In [ ]:
np.unique(vwfa)

(227021, 2)

In [ ]:
lhsurf.coordinates.shape

In [ ]:
vwfa1seed.shape

In [ ]:
        vwfa1seed = np.nanmean(lhbold[vwfa==2],axis=0)
        vwfa2seed = np.nanmean(lhbold[vwfa==3],axis=0)

In [ ]:
vwfa1con = np.zeros(lhbold.shape[0])
vwfa2con = np.zeros(lhbold.shape[0])
for v in range(lhbold.shape[0]):
    vwfa1rval[v]=np.corrcoef(np.transpose(vwfa1seed), np.transpose(lhbold[v]))[0,1]
    vwfa2rval[v]=np.corrcoef(np.transpose(vwfa2seed), np.transpose(lhbold[v]))[0,1]

In [ ]:
plotting.plot_surf_stat_map(lhsurf, stat_map=vwfa1con, bg_map=curv , threshold = 0.1, vmax = .5,view = 'ventral', cmap = 'jet',colorbar=True)

In [ ]:
vwfa1seed.shape